In [ ]:
import numpy as np
import pymultinest

import matplotlib as mpl
import matplotlib.pyplot as plt
#%matplotlib inline

import json

import corner

import retrieval_base.auxiliary_functions as af

from petitRADTRANS import Radtrans
from petitRADTRANS import nat_cst as nc

import petitRADTRANS.poor_mans_nonequ_chem as pm


# import species

In [ ]:
posterior_2, bestfit_2, PT_2, Chem_2, int_contr_em_2, int_contr_em_per_order_2, int_opa_cloud_2, m_spec_2, d_spec_2, LogLike_2, Cov_2, bestfit_params_2 = res

In [ ]:
pRT_atm_2

In [ ]:

# Compute the VMR profiles
species_pRT_names = {
    'CO': 'CO_12_HITEMP_R_300', 
    '13CO': 'CO_13_HITEMP_R_300', 
    'H2O': 'H2O_Exomol_R_300', 
    'CH4': 'CH4_R_300', 
    'NH3': 'NH3_R_300', 
    'CO2': 'CO2_R_300', 
    'HCN': 'HCN_R_300', 
    'H2': 'H2', 
    'He': 'He', 
    ####
    'FeH': 'FeH_R_300', 
    'H2S': 'H2S_R_300', 
    'VO': 'VO_R_300', 
    'TiO': 'TiO_all_Exomol_R_300', 
    'K': 'K_allard_R_300', 
    'Na': 'Na_allard_R_300', 
    }


CO = params['C/O']
FeH = params['Fe/H']
P_quench = params['P_quench']
C_ratio = params['C_ratio']

pm_mass_fractions = pm.interpol_abundances(
    CO*np.ones_like(pressure), 
    FeH*np.ones_like(pressure), 
    temperature, 
    pressure
    )


for i, species_i in enumerate(['CO', 'CH4', 'H2O']):

    line_species_i = species_pRT_names[species_i]
    mass_fraction_i = pm_mass_fractions[species_i]
    pm_mass_fractions[species_i] = mass_fraction_i

mass_fractions = {
    'MMW': pm_mass_fractions['MMW'], 

    'CO_12_HITEMP_R_300': (1 - C_ratio*(13+16)/(12+16)) * pm_mass_fractions['CO'], 
    'CO_13_HITEMP_R_300': C_ratio*(13+16)/(12+16) * pm_mass_fractions['CO'], 

    'H2O_Exomol_R_300': pm_mass_fractions['H2O'], 
    'CH4_R_300': pm_mass_fractions['CH4'], 
    'NH3_R_300': pm_mass_fractions['NH3'], 
    'CO2_R_300': pm_mass_fractions['CO2'], 
    'HCN_R_300': pm_mass_fractions['HCN'], 
    'H2': pm_mass_fractions['H2'], 
    'He': pm_mass_fractions['He'], 
    ####
    'FeH_R_300': pm_mass_fractions['FeH'], 
    'H2S_R_300': pm_mass_fractions['H2S'], 
    'VO_R_300': pm_mass_fractions['VO'], 
    'TiO_all_Exomol_R_300': pm_mass_fractions['TiO'], 
    'K_allard_R_300': pm_mass_fractions['K'], 
    'Na_allard_R_300': pm_mass_fractions['Na'], 
    }



In [ ]:
pRT_atm = Radtrans(
    line_species=[
        'CO_12_HITEMP_R_300', 
        'CO_13_HITEMP_R_300', 
        'H2O_Exomol_R_300', 
        'CH4_R_300', 
        'NH3_R_300', 
        'CO2_R_300', 
        'HCN_R_300', 
        ####
        'FeH_R_300', 
        'H2S_R_300', 
        'VO_R_300', 
        'TiO_all_Exomol_R_300', 
        'K_allard_R_300', 
        'Na_allard_R_300', 
        ], 
    rayleigh_species=['H2', 'He'], 
    continuum_opacities=['H2-H2', 'H2-He'], 
    wlen_bords_micron=[0.3,15], 
    )

pRT_atm.setup_opa_structure(PT_2.pressure)

In [ ]:
from tqdm import tqdm
def get_flux_posterior(bestfit_params, posterior, PT, Chem):

    flux_posterior = []
    scaled_flux_posterior = []

    params = bestfit_params.copy()

    for i, sample in enumerate(tqdm(posterior[:])):
        
        params['R_p'] = sample[8]
        params['log_g'] = sample[9]
        params['epsilon_limb'] = sample[10]
        params['vsini'] = sample[11]
        params['rv'] = sample[12]
        
        params['opa_base_gray'] = 10**sample[13]
        params['P_base_gray'] = 10**sample[14]
        params['f_sed_gray'] = sample[15]

        params['C/O'] = sample[16]
        params['Fe/H'] = sample[17]
        params['P_quench'] = 10**sample[18]
        params['C_ratio'] = 10**sample[19]

        params['T_0'] = sample[21]
        params['T_1'] = sample[22]
        params['T_2'] = sample[23]
        params['T_3'] = sample[24]
        params['T_4'] = sample[25]
        params['T_5'] = sample[26]
        params['T_6'] = sample[27]
        
        params['d_log_P_01'] = sample[28]
        params['log_P_knots'] = np.array([
            -6.0, 
            2.0-params['d_log_P_45']-params['d_log_P_34']-params['d_log_P_23']-params['d_log_P_12']-params['d_log_P_01'], 
            2.0-params['d_log_P_34']-params['d_log_P_23']-params['d_log_P_12']-params['d_log_P_01'], 
            2.0-params['d_log_P_23']-params['d_log_P_12']-params['d_log_P_01'], 
            2.0-params['d_log_P_12']-params['d_log_P_01'], 
            2.0-params['d_log_P_01'], 
            2.0
            ])
        params['P_knots'] = 10**params['log_P_knots']
        
        params['T_knots'] = np.array([
            params['T_6'], 
            params['T_5'], 
            params['T_4'], 
            params['T_3'], 
            params['T_2'], 
            params['T_1'], 
            #params['T_0'], 
            ])


        # Compute the PT profile
        temperature = PT(params)

        
        mass_fractions, _ = ####

        pRT_atm_2.params = params

        # Generate the pRT model spectrum 
        pRT_atm.calc_flux(
            temperature, 
            mass_fractions, 
            gravity=10**params['log_g'], 
            mmw=mass_fractions['MMW'], 
            give_absorption_opacity=pRT_atm_2.gray_cloud_opacity, 
            )
        wave = nc.c / pRT_atm.freq
        flux = pRT_atm.flux

        # Convert [erg cm^{-2} s^{-1} Hz^{-1}] -> [erg cm^{-2} s^{-1} cm^{-1}]
        flux *= nc.c / (wave**2)

        # Convert [erg cm^{-2} s^{-1} cm^{-1}] -> [erg cm^{-2} s^{-1} nm^{-1}]
        flux /= 1e7

        # Convert [erg cm^{-2} s^{-1} nm^{-1}] -> [W cm^{-2} nm^{-1}]
        flux *= 1e-7

        # Convert [W cm^{-2} nm^{-1}] -> [W m^{-2} nm^{-1}]
        flux /= (1e-2)**2

        # Convert [W m^{-2} nm^{-1}] -> [W m^{-2} um^{-1}]
        flux /= 1e-3

        # Convert [cm] -> [nm]
        #wave *= 1e7

        # Convert [cm] -> [um]
        wave *= 1e4

        flux_posterior.append(flux)

        # Convert to observation by scaling with planetary radius
        scaled_flux_posterior.append(flux*(params['R_p']*nc.r_jup_mean / (1e3/params['parallax']*nc.pc))**2)

    return wave, np.array(flux_posterior), np.array(scaled_flux_posterior)

wave, flux_posterior, scaled_flux_posterior = get_flux_posterior(bestfit_params_2['params'], posterior_2, PT_2, Chem_2)